In [1]:
pip install sendgrid

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import time
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

# Adding testing video path
cam = cv2.VideoCapture('Testing.mp4')

# Initializing Background Subtractor
bg = cv2.createBackgroundSubtractorMOG2()
j = 0

# SendGrid Email credentials
SENDGRID_API_KEY = '#'  
FROM_EMAIL = '#'      
TO_EMAIL = '#'   

def send_email_notification():
    message = Mail(
        from_email=FROM_EMAIL,
        to_emails=TO_EMAIL,
        subject='Fall Detected',
        plain_text_content='Kid has fallen. Please check immediately.'
    )

    try:
        sg = SendGridAPIClient(SENDGRID_API_KEY)
        response = sg.send(message)
        print(f"Email sent successfully, status code: {response.status_code}")
    except Exception as e:
        print(f"Error sending email: {e}")

# Main function
while(1):
    # Reading the frames from the Video Streaming
    ret, frame = cam.read()
    if not ret:
        break  # Exit if video ends
    
    # Gray scaling of detected frames
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fg = bg.apply(gray)
    
    # Initializing Contours
    contours, _ = cv2.findContours(fg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    if contours:
        areas = []
        for contour in contours:
            ar = cv2.contourArea(contour)  # Defining Contour Area
            areas.append(ar)  # Appending the Area
        
        max_area = max(areas or [0])
        max_area_index = areas.index(max_area)
        cnt = contours[max_area_index]
        M = cv2.moments(cnt)
        
        # Drawing the bounding box to the frames
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.drawContours(fg, [cnt], 0, (255,255,255), 3, maxLevel=0)
        
        if h < w:
            j += 1
        
        # Conditions for fall detection (contour area)
        if j > 20:
            cv2.putText(frame, 'FALL', (x+5, y-5), cv2.FONT_HERSHEY_TRIPLEX, 2, (79,244,255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
            
            # Send Email notification
            send_email_notification()

            # Optionally, delay to prevent multiple notifications
            time.sleep(5)  # Wait for 5 seconds before allowing another detection
            
        if h > w:
            j = 0 
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow('video', frame)
        
        # Exit loop when 'ESC' key is pressed
        if cv2.waitKey(33) == 27:
            break

cv2.destroyAllWindows()


Email sent successfully, status code: 202
Email sent successfully, status code: 202
Email sent successfully, status code: 202
Email sent successfully, status code: 202
Email sent successfully, status code: 202
